# How to transform collection data to Linked Art with linked_art.py

## Introduction

The `linked_Art.py` file transforms collection data in an intermediateJSON format, to Linked Art JSON-LD. 


A .yaml file is used for script variables. The filepath for the .yaml file is specified as a script argument --config

## Example settings.yaml  file

A settings.yaml file is specified in the script argument --config. 

An example config file can be viewed at [example settings.yaml](/digitalobject/settings.yaml)

Example content:

```yaml
- - - # settings for digital object
- settings:
  default_lang        : en
  base_url            : https://data.participatory-archives.ch/
  pia_api_uri         : https://json.participatory-archives.ch/api/v1/images
  pia_api_include     : include=collections,date,place
  page_size           : 500
  directory           : digitalobject/
  a_collection        : data/a_collection
  b_mapped            : data/b_mapped
  c_linked_art        : data/c_linked_art
  template            : template.jsonnet
...
```


## Files
- linked_art.py - main script to transform collection data to Linked Art
- patterns.py - functions representing patterns for collection data in Linked Art 
- settings.py - use function 'query_api()' to check that the correct script arguments have been provided
- digitalobject/data/a_collection/*.json - collection data files from PIA JSON API
- digitalobject/data/b_mapped/*.json - intermediate JSON file to transform to Linked Art JSON-LD
- digitalobject/data/c_linked_art/*.json - output JSON files

## Help
The following will provide information about arguments to use with the script.
```
python3 map_data.py -h 
```

## Example command

```python
map_data.py --objtype <objecttype> --config <config-file> --template <jsonnet-template-filepath>
```
e.g.
```python
python3 map_data.py --objtype set --config set/settings.yaml --template set/template.jsonnet

```

`--objtype` is one of:
- `humanmadeobject`
- `digitalobject` 
- `set`


## Import Python libraries

In [1]:
from cromulent.model import factory
import os
from os import walk
import json
import yaml
from yaml.loader import SafeLoader


from pathlib import Path

import patterns



## Import settings and set local variables

In [4]:

[objtype,config_file] = ['digitalobject', 'digitalobject/settings.yaml']

with open(config_file) as f:
    data = yaml.load(f, Loader=SafeLoader)
    config = data[1]


# dir to store collection data files
directory               = config["directory"]
b_mapped                = directory + config["b_mapped"]
c_linked_art            = directory + config["c_linked_art"]
factory.default_lang    = config["default_lang"]
factory.base_url        = config["base_url"]

## Read in global variables

In [5]:
globalvars = []
# read in config
with open('../globalvars.yaml') as f:
    data = yaml.load(f, Loader=SafeLoader)
    globalvars = data


## Iterate through intermediate JSON files in b_mapped and add filenames to list 

In [6]:
files = []
for (dirpath, dirnames, filenames) in walk(b_mapped):
    files.extend(filenames)
    break
files.sort()

## Iterate over JSON files in b_mapped and generate Linked Art JSON-LD

In [13]:
# iterate over files in b_mapped

from IPython.core.display import HTML

for file in files:
    filename = os.path.basename(file)

    
    with open(b_mapped + '/' + file) as json_file:
        data = json.load(json_file)
        if objtype == "digitalobject":
            data = patterns.digital_object_pattern(data, globalvars)
        elif objtype == "humanmadeobject":
            data = patterns.humanmadeobject_pattern(data, globalvars)
        elif objtype == "set":
            data = patterns.set_pattern(data, globalvars)

        la = factory.toString(data, compact=False)

        # write linked art to file
        f = open(c_linked_art + '/' + filename, "w")
        f.write(la)
        f.close()
    display(HTML("<a target='_new' href='digitalobject/data/c_linked_art/" + filename  + "'>" + filename + "</a>"))